In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('E:/College/Analytics/Python/database.sqlite')

In [3]:
#sqlite_master is a table with database schema
pd.read_sql(""" SELECT *
                FROM sqlite_master
                WHERE type='table';""",
           conn)

,type,name,tbl_name,rootpage,sql
0,table,station,station,2,CREATE TABLE station (\n id INTEGER PRIMARY...
1,table,status,status,3,"CREATE TABLE status (\n station_id INTEGER,..."
2,table,trip,trip,4,CREATE TABLE trip (\n id INTEGER PRIMARY KE...
3,table,weather,weather,5,"CREATE TABLE weather (\n date TEXT,\n ma..."


In [4]:
pd.read_sql('''SELECT *  FROM station LIMIT 10; ''', con=conn)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


In [5]:
# Total docks and station with average capacity of each City
pd.read_sql(''' SELECT city, 
    SUM(dock_count) AS Total_capacity, 
    COUNT(name) AS Station_count, 
    ROUND(SUM(dock_count)/COUNT(name), 2) AS Average_capacity
    FROM station
    GROUP BY city
    ORDER BY station_count DESC;
''', con=conn)

,city,Total_capacity,Station_count,Average_capacity
0,San Francisco,665,35,19.0
1,San Jose,264,16,16.0
2,Mountain View,117,7,16.0
3,Redwood City,115,7,16.0
4,Palo Alto,75,5,15.0


In [6]:
# Docks and Station count per city and date
pd.read_sql('''
    SELECT city,
           CASE
           -- m/d/yyyy
           WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
           -- m/dd/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
           -- mm/d/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
           -- mm/dd/yyyy
           WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/') 
           THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
           ELSE installation_date
           END AS installed_date, 
           SUM(dock_count) AS total_dock_ct,
           COUNT(name) AS station_count
    FROM station
    GROUP BY 1,2
    ORDER BY 2,1;
''', con=conn)

,city,installed_date,total_dock_ct,station_count
0,San Jose,2013-08-05,83,5
1,San Jose,2013-08-06,87,5
2,San Jose,2013-08-07,64,4
3,Redwood City,2013-08-12,60,4
4,Palo Alto,2013-08-14,64,4
5,Mountain View,2013-08-15,46,2
6,Palo Alto,2013-08-15,11,1
7,Redwood City,2013-08-15,40,2
8,Mountain View,2013-08-16,45,3
9,San Francisco,2013-08-19,94,6


In [7]:
#Installment 
pd.read_sql('''
    WITH t1 AS (SELECT city,
                       CASE
                       -- m/d/yyyy
                       WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
                       -- m/dd/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
                       -- mm/d/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
                       -- mm/dd/yyyy
                       WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
                       ELSE installation_date
                       END AS installed_date, 
                       SUM(dock_count) AS total_dock_ct,
                       COUNT(name) AS station_count
                FROM station
                GROUP BY 1,2
                ORDER BY 2,1)
    SELECT CASE 
        WHEN month = '2013-08-01'
        THEN 'AUG13'
        WHEN month > '2013-08-01'
        THEN 'after_AUG13'
        ELSE 'before_AUG13'
        END AS installation_month,
        SUM(total_dock_ct) AS dock_ct,
        SUM(station_count) AS station_ct
    FROM (SELECT DATE(installed_date, 'start of month') as month,
                 total_dock_ct, 
                 station_count
         FROM t1
         ) AS innerquery
    GROUP BY 1;
''', con=conn, index_col='installation_month')

,dock_ct,station_ct
installation_month,,
AUG13,1150,64
after_AUG13,86,6


In [8]:
pd.read_sql('''
     SELECT * 
     FROM status
     LIMIT 10;
''', con=conn)

,station_id,bikes_available,docks_available,time
0,2,2,25,2013/08/29 12:06:01
1,2,2,25,2013/08/29 12:07:01
2,2,2,25,2013/08/29 12:08:01
3,2,2,25,2013/08/29 12:09:01
4,2,2,25,2013/08/29 12:10:01
5,2,2,25,2013/08/29 12:11:01
6,2,2,25,2013/08/29 12:12:01
7,2,2,25,2013/08/29 12:13:01
8,2,2,25,2013/08/29 12:15:01
9,2,2,25,2013/08/29 12:16:02


In [ ]:
#Bikes availablity vs docks
pd.read_sql('''
     SELECT ROUND(AVG(status.bikes_available),2) AS avg_available_bikes, 
            ROUND(AVG(status.docks_available),2) AS free_dock_count,
            station.dock_count AS max_dock_capacity,
            station.name
     FROM status
     INNER JOIN station
     ON status.station_id = station.id
     GROUP BY name
     ORDER BY 1, 2 DESC
     LIMIT 10;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT ROUND(AVG(status.bikes_available),2) AS avg_available_bikes, 
           ROUND(AVG(status.docks_available),2) AS free_dock_count,
           COUNT(*) AS num_occurrences,
           DATE(SUBSTR(time, 1,4) || '-' || SUBSTR(time, 6,2) || '-' || 
                SUBSTR(time, 9,2) ||  SUBSTR(time, 11,9), 'start of month') AS month 
    FROM status
    INNER JOIN station
    ON status.station_id = station.id
    WHERE station.name = '2nd at Folsom'
    GROUP BY station.name, month
    ORDER BY 2 DESC;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT *
    FROM trip
    LIMIT 10
''', con=conn)

In [ ]:
# Trip Duration Analysis
pd.read_sql('''
    SELECT duration/60/60 AS duration_hr, COUNT(*) AS frequency
    FROM trip
    GROUP BY duration_hr
    ORDER BY duration_hr DESC 
    LIMIT 15;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT duration/60/60 AS duration_hr, COUNT(*) AS frequency
    FROM trip
    WHERE duration_hr <= 15
    GROUP BY duration_hr
    ORDER BY duration_hr DESC;
''', con=conn)

In [ ]:
#Busiest pickup and dropoff locations
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           trip.start_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco'
    GROUP BY 2
    ORDER BY 1 DESC
    LIMIT 5;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT COUNT(*) AS num_count, 
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco'
    GROUP BY 2
    ORDER BY 1 DESC
    LIMIT 5;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco' AND duration/60/60 <= 15
    GROUP BY 3, 4
    ORDER BY 1 DESC
    LIMIT 15;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT AVG(avg_duration_mins) AS avg_duration_for_most_pop
    FROM (SELECT COUNT(*) AS num_count,
                 ROUND(AVG(duration/60), 2) AS avg_duration_mins,
                 trip.start_station_name,
                 trip.end_station_name
          FROM trip
          INNER JOIN station
          ON station.id = start_station_id
          WHERE station.city = 'San Francisco' AND duration/60/60 <= 15
          GROUP BY 3, 4
          HAVING COUNT(*) > 2000
          ORDER BY 1 DESC) AS most_popular_trips
''', con=conn)

In [ ]:
#Popular long-trip routes 
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE city='San Francisco' AND duration/60/60 <= 15
    GROUP BY 3, 4
    HAVING COUNT(*) > 50
    ORDER BY 2 DESC
    LIMIT 15;
''', con=conn)

In [ ]:
pd.read_sql('''
    SELECT subscription_type,
           COUNT(*) AS sub_type_ct
    FROM trip
    GROUP BY 1;
''', con=conn)

In [ ]:
#Average duration per subscriber type
pd.read_sql('''
    WITH t1 AS (SELECT DATE(CASE
                -- m/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 8 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,5,4)||'-0'||substr(start_date,1,1)||'-0'||substr(start_date,3,1)
               -- m/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,1)||'-'||substr(start_date,3,2)
               -- mm/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,2)||'-'||substr(start_date,4,1)
               -- mm/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 10 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,7,4)||'-'||substr(start_date,1,2)||'-'||substr(start_date,4,2)
               ELSE start_date
               END) AS trip_date,
               subscription_type, 
               (duration / 60) AS duration_min
        FROM trip
        INNER JOIN station
        ON station.id = start_station_id
        WHERE city='San Francisco' AND duration/60/60 <= 15)
    SELECT CASE 
           WHEN (trip_date IN (DATE(trip_date, 'weekday 6'), DATE(trip_date, 'weekday 0')))
           THEN 'weekends'
           ELSE 'weekdays'
           END AS weekday,
           subscription_type,
           ROUND(AVG(duration_min), 2) AS avg_dur_min
           
    FROM t1
    GROUP BY 1,2;
''', con=conn)

In [ ]:
pd.read_sql('''
    WITH t1 AS (SELECT DATE(CASE
                -- m/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 8 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,5,4)||'-0'||substr(start_date,1,1)||'-0'||substr(start_date,3,1)
               -- m/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,1)||'-'||substr(start_date,3,2)
               -- mm/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,2)||'-'||substr(start_date,4,1)
               -- mm/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 10 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,7,4)||'-'||substr(start_date,1,2)||'-'||substr(start_date,4,2)
               ELSE start_date
               END) AS trip_date,
               subscription_type, 
               (duration / 60) AS duration_min
        FROM trip
        INNER JOIN station
        ON station.id = start_station_id
        WHERE city='San Francisco' AND duration/60/60 <= 15)
    SELECT CASE 
           WHEN trip_date = (DATE(trip_date, 'weekday 1'))
           THEN '1 - Monday'
           WHEN trip_date = (DATE(trip_date, 'weekday 2'))
           THEN '2 - Tuesday'
           WHEN trip_date = (DATE(trip_date, 'weekday 3'))
           THEN '3 - Wednesday'
           WHEN trip_date = (DATE(trip_date, 'weekday 4'))
           THEN '4 - Thursday'
           WHEN trip_date = (DATE(trip_date, 'weekday 5'))
           THEN '5 - Friday'
           WHEN trip_date = (DATE(trip_date, 'weekday 6'))
           THEN '6 - Saturday'
           ELSE '7 - Sunday'
           END AS weekday,
           subscription_type,
           ROUND(AVG(duration_min), 2) AS avg_dur_min
           
    FROM t1
    GROUP BY 1,2
    ORDER BY 2,1;
''', con=conn)

In [ ]:
#Popular locations and average duration time
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco' AND duration/60/60 <= 15 AND subscription_type = 'Customer'
    GROUP BY 3, 4
    ORDER BY 1 DESC
    LIMIT 15;
''', con=conn)